In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
from fastai.text.all import *

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
get_imdb = partial(get_text_files,folders=['train','test','unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [5]:
len(dls_lm.vocab)

60008

In [8]:
dls_lm.vocab[0:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'the']

In [13]:
print(dls_lm)

In [11]:
dls_lm.show_batch()

,text,text_
0,"xxbos i used to love xxmaj sabrina xxmaj the xxmaj teenage xxmaj witch and have seen every single episode . i remember when i used to sit at 6 pm every night and wait for it to come on xxmaj nickelodeon , however when xxmaj sabrina left high school the show began to go downhill . xxmaj the best series has to be when she was friends with xxmaj valerie ( i 'm not sure which one that is )","i used to love xxmaj sabrina xxmaj the xxmaj teenage xxmaj witch and have seen every single episode . i remember when i used to sit at 6 pm every night and wait for it to come on xxmaj nickelodeon , however when xxmaj sabrina left high school the show began to go downhill . xxmaj the best series has to be when she was friends with xxmaj valerie ( i 'm not sure which one that is ) ."
1,"problems to survive they own destruction . xxmaj as i said before , the show began great , all was almost perfect , including characters like xxmaj da'an , the original xxmaj da'an was a big mystery because he seems to be a nice creature but at the same time he has his own evil plans manipulating some people in earth . xxmaj soon came xxmaj xxunk who was n't bad but … mark the beginning of the fall of","to survive they own destruction . xxmaj as i said before , the show began great , all was almost perfect , including characters like xxmaj da'an , the original xxmaj da'an was a big mystery because he seems to be a nice creature but at the same time he has his own evil plans manipulating some people in earth . xxmaj soon came xxmaj xxunk who was n't bad but … mark the beginning of the fall of this"
2,"have changed since now most xxup tv sitcoms are about young people and the industry avoids most shows about middle - aged people like the plague ! ) \n\n xxmaj subsequently , one of the young stars of "" love in the xxmaj happy xxmaj days "" , a child actor from "" the xxmaj andy xxmaj griffith xxmaj show "" named xxmaj ron xxmaj howard , got the chance to star in a film about young people taking place","changed since now most xxup tv sitcoms are about young people and the industry avoids most shows about middle - aged people like the plague ! ) \n\n xxmaj subsequently , one of the young stars of "" love in the xxmaj happy xxmaj days "" , a child actor from "" the xxmaj andy xxmaj griffith xxmaj show "" named xxmaj ron xxmaj howard , got the chance to star in a film about young people taking place in"
3,"face this opinion in a discussion and if you are willing to really look into the art of xxunk you would have to "" surrender "" . xxbos xxmaj this cordial comedy confronts a few bizarre characters . xxmaj especially , of course , the two leading characters . xxmaj jack xxmaj lemmon plays xxmaj felix , a hypochondriac whose wife lost him because she could n't stand his cleaning and cooking attacks any longer . xxmaj so he tries","this opinion in a discussion and if you are willing to really look into the art of xxunk you would have to "" surrender "" . xxbos xxmaj this cordial comedy confronts a few bizarre characters . xxmaj especially , of course , the two leading characters . xxmaj jack xxmaj lemmon plays xxmaj felix , a hypochondriac whose wife lost him because she could n't stand his cleaning and cooking attacks any longer . xxmaj so he tries to"
4,"and when the accents was so poor , this equaled in very low performance , and i sometimes felt that i was watching xxmaj swedish xxmaj big xxmaj brother ▁ i think this is because xxmaj swedish actors has not yet understood that acting is with whole body , eye moves , body - language , etc , not just standing there like a jukebox and saying your lines one after other . xxmaj the kids acting was horrible ,","when the accents was so poor , this equaled in very low performance , and i sometimes felt that i was watching xxmaj swedish xxmaj big xxmaj brother ▁ i think this is because xxmaj swedish actors has not yet understood that acting is with whole body , eye moves , body - language , etc , not just stan

In [31]:
def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / np.sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))
        return attn_outputs
    
class MultiHeadAttention(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads):
        super().__init__()
        embed_dim = embed_dim
        num_heads = num_heads
        head_dim = embed_dim // num_heads
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, vocab_size)

    def forward(self, hidden_state):
        #print(hidden_state)
        embed_state = self.embedding(hidden_state)
        #print(embed_state)
        x = torch.cat([h(embed_state) for h in self.heads], dim=-1)
        #print("1")
        x = self.output_linear(x)
        print(x)
        return x

In [ ]:
model = MultiHeadAttention(
    vocab_size=len(dls_lm.vocab), 
    embed_dim=256, 
    num_heads=4
)

learn = Learner(
    dls_lm, 
    model, 
    loss_func=CrossEntropyLossFlat(), 
    metrics=[accuracy]
)

learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time


tensor([[[-0.0285,  0.0580, -0.0790,  ...,  0.0286, -0.2310,  0.0477],
         [ 0.0188,  0.0604, -0.0434,  ...,  0.0793, -0.1644,  0.0219],
         [-0.0388,  0.0452, -0.0554,  ...,  0.0473, -0.2214,  0.0319],
         ...,
         [-0.0038,  0.0499, -0.0140,  ...,  0.0348, -0.1784,  0.0079],
         [-0.0396,  0.0488, -0.0903,  ...,  0.0321, -0.2058,  0.0352],
         [ 0.0019,  0.0836, -0.0469,  ...,  0.0432, -0.2038,  0.0524]],

        [[-0.0167,  0.0644, -0.0180,  ...,  0.0677, -0.1205,  0.0559],
         [ 0.0192,  0.0097, -0.0373,  ...,  0.0223, -0.1233,  0.0263],
         [-0.0225,  0.0499, -0.0296,  ..., -0.0113, -0.1567,  0.0819],
         ...,
         [-0.0249,  0.0673, -0.0459,  ..., -0.0296, -0.1457,  0.0724],
         [-0.0362, -0.0026, -0.0464,  ...,  0.0763, -0.1917,  0.0577],
         [-0.0167,  0.0644, -0.0180,  ...,  0.0677, -0.1205,  0.0559]],

        [[-0.0333,  0.1203, -0.0801,  ...,  0.1295, -0.0743, -0.0215],
         [-0.0087,  0.1030, -0.0722,  ...,  0

tensor([[[ 0.0271,  0.0085, -0.1118,  ...,  0.0740, -0.0515, -0.0342],
         [-0.0331,  0.0293, -0.0969,  ...,  0.0877, -0.0111, -0.0356],
         [-0.0073, -0.0325, -0.0941,  ...,  0.0781, -0.0434, -0.0731],
         ...,
         [-0.0265,  0.0131, -0.0855,  ...,  0.0789, -0.0647, -0.0294],
         [-0.0322,  0.0041, -0.0920,  ...,  0.0409, -0.0540, -0.0059],
         [-0.0220, -0.0254, -0.0779,  ...,  0.0894, -0.0767, -0.0489]],

        [[ 0.0374,  0.0106, -0.0964,  ...,  0.1169, -0.0588, -0.0137],
         [ 0.0085,  0.0387, -0.0900,  ...,  0.1208, -0.0946, -0.0004],
         [ 0.0182, -0.0030, -0.0699,  ...,  0.1550, -0.1156, -0.0147],
         ...,
         [ 0.0185, -0.0186, -0.0713,  ...,  0.0864, -0.0846, -0.0240],
         [ 0.0514,  0.0221, -0.0798,  ...,  0.1008, -0.0808,  0.0011],
         [ 0.0089,  0.0171, -0.0629,  ...,  0.1039, -0.0847, -0.0095]],

        [[ 0.0188, -0.0116, -0.0762,  ...,  0.0459, -0.0337,  0.0356],
         [ 0.0695, -0.0349, -0.0790,  ...,  0

tensor([[[-1.4582e-02,  9.1344e-02, -1.0687e-01,  ..., -1.0206e-02,
          -1.2818e-01, -1.5463e-03],
         [-1.8912e-02,  4.0938e-02, -8.7556e-02,  ..., -2.4949e-03,
          -1.3941e-01, -4.4411e-03],
         [-9.4666e-04,  5.7853e-02, -8.3874e-02,  ...,  1.8113e-02,
          -1.0705e-01, -7.6069e-03],
         ...,
         [-2.4028e-02,  6.3673e-02, -8.2221e-02,  ...,  1.4051e-02,
          -1.3951e-01, -3.4789e-03],
         [-2.7964e-02,  4.2615e-02, -6.9538e-02,  ...,  6.0354e-03,
          -1.4031e-01, -2.2036e-02],
         [-3.6167e-02,  8.7227e-02, -8.9230e-02,  ..., -1.4861e-02,
          -1.4033e-01,  6.8436e-03]],

        [[ 4.3593e-02, -2.5808e-03, -5.2003e-02,  ...,  1.1877e-01,
          -6.7161e-02,  4.4058e-02],
         [-1.5876e-02, -1.0962e-02, -5.5414e-02,  ...,  1.0821e-01,
          -9.6247e-02,  7.5857e-02],
         [ 3.9008e-02, -9.9646e-03, -8.8443e-02,  ...,  1.0175e-01,
          -6.6031e-02,  3.5053e-02],
         ...,
         [-4.5003e-04, -2

In [26]:
import torch
import torch.nn as nn

# Example input and target tensors
input = torch.randn(3, 5)  # Input tensor with shape (batch_size, num_classes)
target = torch.tensor([1, 0, 4])  # Target tensor with shape (batch_size)

# Create an instance of CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# Compute the cross-entropy loss
loss = loss_fn(input, target)

print(loss)


tensor(2.6823)
